# Model Creation

## Import Libraries

In [2]:
import os
from operator import itemgetter    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
# from sklearn.preprocessing import Imputer
# from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder, LabelBinarizer
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
# from sklearn.cross_validation import KFold, cross_val_score
# Cross Validation, use the method in XGBoost if needed. 

from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, KFold, cross_val_predict, StratifiedKFold, train_test_split, learning_curve, ShuffleSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

import tensorflow as tf

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils



### Import Data from Pre-Processing

In [3]:
df= pd.read_csv('./_data/1hot_encoded.csv')
df.shape
df.info

<bound method DataFrame.info of        Unnamed: 0   age  weight  height     art_mbp     art_sbp         bt  \
0               3  50.0    66.0   157.0  100.826720  137.244265  29.656043   
1               4  60.0    62.0   154.0  100.773779  138.601562  28.462637   
2               5  35.0    50.0   160.0   98.084871  131.646617  29.201952   
3               6  20.0    62.0   179.0   92.702290  140.826772  35.800000   
4               7  60.0    52.0   152.0  100.773779  138.601562  28.462637   
...           ...   ...     ...     ...         ...         ...        ...   
74778       87838  80.0    61.0   171.0  106.000000  172.000000  27.672313   
74779       87839  55.0    51.0   162.0  102.797362  139.350856  29.258995   
74780       87840  80.0    60.0   160.0   99.704409  146.300403  27.672313   
74781       87841  30.0    72.0   176.0  134.000000  178.000000  29.013889   
74782       87842  45.0    58.0   158.0   98.192708  134.999118  30.278016   

            cvp          hr    

### Create the X and y DataFrames

  * create y
  * create X (complete with all the features)
  * drop the features we identified as not meeting impact threshold. 
  * 


In [4]:
# Label = LOS

features_to_retain = ['asa_3.0',	'asa_2.0',	'weight',	'age',	'height',		'anesth_duration',	'or_duration',	'icu_visit',	'department_OT',	'antype_MAC',	'antype_Neuraxial',	'department_OS',	'department_GS',	'department_OG',	'department_OL',	'department_NS',	'department_CTS',	'department_UR',	'ast',	'alt',	'total_bilirubin',	'lymphocyte',	'wbc',	'art_sbp',	'pmean',	'art_mbp',	'vt',	'pip',	'rr',	'hr',	'chloride',	'hb',	'sodium',	'hco3',	'platelet',	'category_id_08D',	'category_id_0HB',	'category_id_09Q',	'category_id_08R',	'category_id_0DB',	'category_id_0QS',	'category_id_0DT',	'category_id_0W3',	'category_id_07L',	'category_id_0FT',	'category_id_0VB',	'category_id_09P'] 

y = df['LOS']
X = df.drop('LOS', axis=1)
X = X[features_to_retain]
X

,asa_3.0,asa_2.0,weight,age,height,anesth_duration,or_duration,icu_visit,department_OT,antype_MAC,...,category_id_09Q,category_id_08R,category_id_0DB,category_id_0QS,category_id_0DT,category_id_0W3,category_id_07L,category_id_0FT,category_id_0VB,category_id_09P
0,0.0,0.0,66.0,50.0,157.0,70.0,45.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,62.0,60.0,154.0,90.0,70.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,50.0,35.0,160.0,150.0,115.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,62.0,20.0,179.0,135.0,90.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,52.0,60.0,152.0,90.0,30.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74778,1.0,0.0,61.0,80.0,171.0,380.0,320.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74779,0.0,1.0,51.0,55.0,162.0,100.0,65.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74780,0.0,1.0,60.0,80.0,160.0,150.0,120.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74781,0.0,1.0,72.0,30.0,176.0,100.0,65.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Original DF
print(df.shape)
# X DF with dropped LOS and non-important featuresd.
print(X.shape)



(74783, 170)
(74783, 47)


### Normalize Y (LOS)

In [6]:
# Normalize y
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

y = y.values #returns a numpy array
y = y.reshape(-1, 1)
y_scaled = scaler.fit_transform(y)
ynorm=pd.DataFrame(y_scaled)
print(ynorm)

              0
0     -0.447474
1     -0.273747
2     -0.150233
3     -0.592920
4     -0.587148
...         ...
74778  1.842721
74779 -0.620624
74780 -0.465943
74781  0.040809
74782 -0.455554

[74783 rows x 1 columns]


In [ ]:
print(ynorm)

## Training - Baseline

### Split data
- Training Set (80% of total): 
  - Used to train the models.
- Validation Set (20% of Traning Set ): 
  - Used to fine-tune hyperparameters, select models, and monitor training progress.  
- Testing Set (20% of total): 
  - Used to evaluate the final model's performance on unseen data and estimate its generalization performance.

In [8]:
from sklearn.model_selection import train_test_split

TEST_SPLIT = .2
TRAINING_SPLIT = 1-TEST_SPLIT
VALIDATION_SPLIT = .2

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, ynorm, test_size=TEST_SPLIT, random_state=85100)

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=TEST_SPLIT, random_state=85100)

# Then, you can use X_train and y_train for model training and X_test and y_test for evaluation.

data_subset_dict = {
    'X_train': X_train,
    'X_validate': X_validate,
    'X_test': X_test,
    'y_train': y_train,
    'y_validate': y_validate,
    'y_test': y_test}

for key, value in data_subset_dict.items():
    shape = value.shape
    print(f"{key} shape: {shape}")


X_train shape: (47860, 47)
X_validate shape: (11966, 47)
X_test shape: (14957, 47)
y_train shape: (47860, 1)
y_validate shape: (11966, 1)
y_test shape: (14957, 1)


### Random Forest



In [9]:
## INITIALIZE MODEL

rfc_100 = RandomForestRegressor(n_estimators=100, random_state=0)

In [ ]:
# fit the model to the training set
rfc_100.fit(X_train, y_train)

In [ ]:
# Predict on the Validation set results
y_pred_100 = rfc_100.predict(X_test)

# Check accuracy score 
print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))

## MODEL PIPELINE

In [10]:
## Define an evaluation function Root Mean Squared Error:

def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model,X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse



In [11]:
#########################
#
#  SIMPLE Pipeline -
#  -- No tuning. 
########################
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest


model_list= [LinearRegression(),ExtraTreesRegressor (),RandomForestRegressor(),XGBRegressor()]
model_names = ["Linear Regression","ExtraTreesRegressor", "Random Forest","XGBRegressor"]

ModScores = {}
for model_names, model in zip(model_names, model_list):
    pipeline = Pipeline(steps=[('select_best', SelectKBest()), # Optional if not already perofrmed above: 'scaling', StandardScaler()),
                           ('classifier', model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_validate)

    score = rmse_cv(model,y_validate, y_pred)
    ModScores[model_names] = score.mean()
    print("{}: {:.2f}".format(model_names,score.mean()))

print("_"*100)
for key, value in sorted(ModScores.items(), key = itemgetter(1), reverse = False):
    print(key, round(value,3))



Linear Regression: 0.43
ExtraTreesRegressor: 0.62
Random Forest: 0.58
XGBRegressor: 0.56
____________________________________________________________________________________________________
Linear Regression 0.429
XGBRegressor 0.557
Random Forest 0.577
ExtraTreesRegressor 0.621


### Optimize Models with Hyperparameter Tuning via Grid Search


In [ ]:
#########################
#
#  Pipeline with TUNING 
# 
########################
from sklearn.linear_model import RandomForestRegressor,XGBRegressor,Ex
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

model_list= [ExtraTreesRegressor (),RandomForestRegressor(),XGBRegressor()]
model_names = ["ExtraTreesRegressor", "Random Forest","XGBRegressor"]

# Find the best hyperparameters using GridSearchCV on the train set
param_grids = {
    'RandomForestRegressor': {
                    'randomforestregressor__n_estimators': [50, 100, 200],
                    'randomforestregressor__max_depth': [None, 10, 20, 30],
                    'randomforestregressor__min_samples_split': [2, 5, 10],
                    'randomforestregressor__min_samples_leaf': [1, 2, 4],
                    'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'],
                    'randomforestregressor__bootstrap': [True, False]
            },          
    'ExtraTreesRegressor':{
                    'extratreesregressor__n_estimators': [50, 100, 200],
                    'extratreesregressor__max_depth': [None, 10, 20, 30],
                    'extratreesregressor__min_samples_split': [2, 5, 10],
                    'extratreesregressor__min_samples_leaf': [1, 2, 4],
                    'extratreesregressor__max_features': ['auto', 'sqrt', 'log2'],
                    'extratreesregressor__bootstrap': [True, False]
                    },
    'XGBRegressor': {
                    'xgb__objective': ['reg:squarederror'],
                    'xgb__tree_method': ['gpu_hist'],
                    'xgb__n_estimators': [50, 100, 200],# Number of boosting rounds (trees)
                    'xgb__learning_rate': [0.01, 0.1, 0.2],# Learning rate
                    'xgb__max_depth': [3, 4, 5],# Maximum depth of each tree
                    'xgb__min_child_weight': [1, 2, 3],# Minimum sum of instance weight (hessian) needed in a child
                    'xgb__subsample': [0.8, 0.9, 1.0],# Subsample ratio of columns when constructing each tree
                    'xgb__colsample_bytree': [0.8, 0.9, 1.0]
                    }
        }

ModScores = {}
for model_name, model in zip(model_names, model_list):
    pipeline = Pipeline(steps=[('select_best', SelectKBest()),
                               ('classifier', model)])
    
     # Access the parameter grid for the current model
    param_grid = param_grids[model_name]
    grid = GridSearchCV(pipeline, param_grid=param_grid, cv=5)
    grid.fit(X_train, y_train)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_validate)

    score = rmse_cv(model,y_validate, y_pred)
    ModScores[model_names] = score.mean()
    print("{}: {:.2f}".format(model_names,score.mean()))

    best_model = grid.best_estimator_
    best_hyperparams = grid.best_params_
    best_acc = grid.score(X_test, y_test)
    print(f'Best test set accuracy: {best_acc}\nAchieved with hyperparameters: {best_hyperparams}')

print("_"*100)
for key, value in sorted(ModScores.items(), key = itemgetter(1), reverse = False):
    print(key, round(value,3))

